Map each observed healthcare task onto an ONET DWA,    
    
- use a string matching function to Task descriptions for assistance. 

By Paul Duckworth Feb 2018.


In [30]:
import os
import numpy as np
import pandas as pd
import getpass
import matplotlib.pyplot as plt
import gensim
import pymed
import time
%matplotlib inline

datasets = '/home/'+ getpass.getuser() +'/Datasets/'
print(datasets)

/home/scpd/Datasets/


## OBSERVED TASK DATA

In [32]:
d1 = os.path.join(datasets, 'FoHealthcare/FOH Occupations Tasks Features Technology.xlsx')
d2 = os.path.join(datasets, 'FoHealthcare/expanded tasks descriptions_for matching DWAs.csv')

excel_doc = pd.ExcelFile(d1)
matts_dataset = excel_doc.parse("Title, Tasks, Features").fillna("-")
matts_dataset.rename(columns = {'Occupation title':'Observed Occupation'}, inplace = True)
matts_dataset['Task ID'] = matts_dataset.index

# Test data with added Task "context".
data = pd.read_csv(d2)
data.rename(columns = {'Occupation title':'Observed Occupation', 'Task keywords/context':'Context'}, inplace = True)
data['Task ID'] = data.index
data

,Observed Occupation,Task,Context,Task ID
0,Administrator,Medical Coding,Structured clinical vocabulary highlight docum...,0
1,Administrator,Write QOF letters or texts,write quality outcomes framework letter on tex...,1
2,Administrator,Mass mail letters for checkups using DocMail,print spreadsheet post mail automated address ...,2
3,Administrator,Scan documents,open mail and take out letter place on scannin...,3
4,Deputy Practice Manager,Talk to doctors about incentive schemes,Talk converse negotiate pros and cons plan for...,4
5,Deputy Practice Manager,Data work in case notes system,read notes understand patients cases outside c...,5
6,General Practitioner,Study patient medical record,research review patient electronic medical rec...,6
7,General Practitioner,Giving advice to colleagues/chatting,impromptu advice unplanned unscheduled meeting...,7
8,General Practitioner,Approve/review RX,type in 4 digit pin to approve a prescription ...,8
9,General Practitioner,Phone consultation,Phone consultations the receptionists will ass...,9


## ONET TASK DATA

In [114]:
# ONET Tasks
onet_tasks = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/Task Statements.txt'), sep='\t')
print "all Tasks = ", onet_tasks.shape
onet_tasks = onet_tasks[['Task ID', 'Task']]

# #Task DWAs (detailed work activitiy code):
taskDWA = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/Tasks to DWAs.txt'), sep='\t')
DWArefs = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/DWA Reference.txt'), sep='\t')
DWA_sup = taskDWA[['Task ID', 'DWA ID']].merge(DWArefs[['DWA ID', 'IWA ID', 'DWA Title']], on=['DWA ID'])

onet_tasks_dwa = onet_tasks[['Task ID', 'Task']].merge(DWA_sup, on=['Task ID'])\
                                                         .sort_values(by='Task ID')\
                                                         .reset_index()\
                                                         .drop('index',axis=1)
print "Merged Tasks = ", onet_tasks_dwa.shape
onet_tasks_dwa.head()


all Tasks =  (19566, 7)
Merged Tasks =  (22838, 5)


,Task ID,Task,DWA ID,IWA ID,DWA Title
0,1,Resolve customer complaints regarding sales an...,4.A.4.a.8.I03.D05,4.A.4.a.8.I03,Resolve customer complaints or problems.
1,2,Monitor customer preferences to determine focu...,4.A.1.a.1.I14.D04,4.A.1.a.1.I14,Conduct opinion surveys or needs assessments.
2,3,Direct and coordinate activities involving sal...,4.A.4.b.4.I12.D03,4.A.4.b.4.I12,"Direct sales, marketing, or customer service a..."
3,4,Determine price schedules and discount rates.,4.A.2.b.4.I01.D06,4.A.2.b.4.I01,Determine pricing or monetary policies.
4,5,Review operational records and reports to proj...,4.A.2.a.4.I11.D06,4.A.2.a.4.I11,Analyze financial records or reports to determ...


## LANGUAGE MODEL

In [98]:
# sentace1 = "hello the president is call rodger"
# sentace2 = "the president has gone to play golf"
# model.wmdistance(sentace1, sentace2)

In [ ]:
location = '/home/'+ getpass.getuser() +'/Software/GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(location, binary=True)

In [117]:
# for obs_task, obs_task_id in zip(data['Context'], data['Task ID'])
#     obs_task

first_obs_task = data['Context'][0].lower()
print first_obs_task

structured clinical vocabulary highlight document edit coding billing standards framework 


In [118]:
A = time.time()
onet_tasks['distance'] = onet_tasks['Task'].apply(lambda col: model.wmdistance(col.lower(), first_obs_task))
print "Time = ", time.time()-A
onet_tasks

Time =  84.5212068558


,Task ID,Task,distance
0,8823,Direct or coordinate an organization's financi...,0.795903
1,8831,Appoint department heads or managers and assig...,0.997163
2,8825,Analyze operations to evaluate performance of ...,0.964108
3,8826,"Direct, plan, or implement policies, objective...",0.932675
4,8827,"Prepare budgets for approval, including those ...",0.805721
5,8824,"Confer with board members, organization offici...",0.884198
6,8836,Implement corrective action plans to solve org...,0.987001
7,8832,"Direct human resources activities, including t...",0.771484
8,8835,Establish departmental responsibilities and co...,0.899442
9,8833,"Preside over or serve on boards of directors, ...",1.075835


In [119]:
onet_tasks.sort_values(by='distance',ascending=False, inplace=True)
onet_tasks

,Task ID,Task,distance
11204,2269,Carve meat.,2.066316
937,7206,Prepare project proposals.,1.793860
13606,9783,Pay small claims.,1.776445
10158,19330,Prepare or serve food trays.,1.776095
13274,2728,Receive or prepare work orders.,1.771730
5095,7569,Assess employee performance.,1.768087
4054,3652,Oversee support staff.,1.724047
1982,20015,Supervise staff members.,1.716640
11221,17592,Take out garbage.,1.706963
11849,8059,Shampoo hair.,1.705833


In [93]:
onet_tasks['test'] = onet_tasks['Task'].apply(lambda col: len(col))
onet_tasks

,Task ID,Task,distance,test
0,8823,Direct or coordinate an organization's financi...,1.355371,135
1,8831,Appoint department heads or managers and assig...,1.542297,85
2,8825,Analyze operations to evaluate performance of ...,1.641313,180
3,8826,"Direct, plan, or implement policies, objective...",1.543943,191
4,8827,"Prepare budgets for approval, including those ...",1.481292,88
5,8824,"Confer with board members, organization offici...",1.590039,130
6,8836,Implement corrective action plans to solve org...,1.550559,83
7,8832,"Direct human resources activities, including t...",1.545618,205
8,8835,Establish departmental responsibilities and co...,1.471131,93
9,8833,"Preside over or serve on boards of directors, ...",1.699969,95


In [110]:
t_min = min([len(t.split()) for t in onet_tasks['Task']])
t_max = max([len(t.split()) for t in onet_tasks['Task']])

( - t_min / float(t_max-t_min))


99.88372093023257